In [1]:
!pip install timm

# Making the necessary imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import timm
from tqdm.notebook import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

from time import sleep

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# Unzipping in file 
(kaggle did it for me but for colab we need to unzip it). 
## Location of folder = root dorectory (here gdrive)

In [4]:
# !unzip /content/drive/MyDrive/ds_dataset.zip -d /content/drive/MyDrive/dataset

# Creating DataFrame of files and classes

In [5]:
import os
def create_df(path):
    df = pd.DataFrame(columns=["Path", "Class"])
    for folder in os.listdir(path): #accessing the file system
        for file in os.listdir(os.path.join(path, folder)):
            df.loc[len(df)] = [os.path.join(path, folder, file), folder] #first element of list is the path to image and second element is the folder name i.e., the class of the image
    return df

In [6]:
PATH = "/kaggle/input/" + "stack-dataset" + "/images" #change accordingly to your file system (root + dataset name + images directory within it)
train_df = create_df(f"{PATH}/train")
val_df = create_df(f"{PATH}/validation")
test_df = create_df(f"{PATH}/final test")

# Encoding the class names to labels

In [7]:
le = LabelEncoder() #standard label encoder from sklearn
le.fit(train_df["Class"]) #fitting onto class column of dataframe
train_df["Class"] = le.transform(train_df["Class"]) #transforming the training dataset
val_df["Class"] = le.transform(val_df["Class"]) #applying same transform on the validation dataset

In [8]:
train_df, val_df

(                                                    Path  Class
 0      /kaggle/input/stack-dataset/images/train/surpr...      6
 1      /kaggle/input/stack-dataset/images/train/surpr...      6
 2      /kaggle/input/stack-dataset/images/train/surpr...      6
 3      /kaggle/input/stack-dataset/images/train/surpr...      6
 4      /kaggle/input/stack-dataset/images/train/surpr...      6
 ...                                                  ...    ...
 26916  /kaggle/input/stack-dataset/images/train/happy...      3
 26917  /kaggle/input/stack-dataset/images/train/happy...      3
 26918  /kaggle/input/stack-dataset/images/train/happy...      3
 26919  /kaggle/input/stack-dataset/images/train/happy...      3
 26920  /kaggle/input/stack-dataset/images/train/happy...      3
 
 [26921 rows x 2 columns],
                                                    Path  Class
 0     /kaggle/input/stack-dataset/images/validation/...      6
 1     /kaggle/input/stack-dataset/images/validation/...      6

In [9]:
# X_train, X_val, y_train, y_val = train_test_split(train_df.drop(['Class'], axis=1), train_df["Class"], random_state=42, stratify=train_df["Class"])
# train_data = pd.merge(X_train, y_train, right_index=True, left_index=True)
# valid_data = pd.merge(X_val, y_val, right_index=True, left_index=True)

In [10]:
train_data = train_df
valid_data = val_df

# Creating the Config for the training

In [11]:
class CFG:
    model_names = ['regnetx_320.tv2_in1k', 'regnetx_160.tv2_in1k', 'wide_resnet50_2.tv2_in1k']
    size = 48
    n_epochs = 20
    lr = 0.001
    weight_decay = 0.001
    momentum = 0.9
    n_classes = 7
    batch_size = 256
    train = True

# Creating our custom dataset

In [12]:
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df["Path"].values
        self.labels = df["Class"].values
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        image = cv2.imread(self.file_names[idx]) #reading the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #making the image in the format required for model BGR -> RGB
        if self.transform:
            augmented = self.transform(image = image) #applying augmentations defined if flag is true
            image = augmented['image']
        label = torch.tensor(self.labels[idx]).long() #converting to torch tensor
        return image, label

# Defining our custom image augmentations based on type of dataset

In [13]:
def get_transform(*, data):
    if data == 'train':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.RandomResizedCrop(CFG.size, CFG.size),
            A.HorizontalFlip(p=0.5),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    elif data == 'valid':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    elif data == 'test':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [14]:
class Model(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = timm.create_model(model, pretrained=True) #creating the model based on name given and loading the weights as well from HuggingFace (Timm)
        print(self.model.default_cfg['classifier']) #to check what the head/classifier is called, varies for models hence needed
        if (self.model.default_cfg['classifier'] == "fc"):
            n_features = self.model.fc.in_features
        elif (self.model.default_cfg['classifier'] == "classifier"):
            n_features = self.model.classifier.in_features
        elif (self.model.default_cfg['classifier'] == "head.fc"):
            n_features = self.model.head.fc.in_features
        elif (self.model.default_cfg['classifier'] == "classif"):
            n_features = self.model.classif.in_features
        self.model.fc = nn.Linear(n_features, CFG.n_classes) 
    def forward(self, x):
        return self.model(x)

# Utility functions for the training process

## Loss function

In [15]:
def get_criterion():
    criterion = nn.CrossEntropyLoss()
    return criterion

## Metric

In [16]:
def get_score(y_true, y_pred):
    check = y_true - y_pred
    return np.count_nonzero(check == 0)/len(check)

## Optimizer

In [17]:
def get_optimizer(model):
    optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    return optimizer

## LR Scheduler

In [18]:
def get_scheduler(optimizer):
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.25, verbose=True)
    return scheduler

# Single epoch training and validation functions

In [19]:
def train_fn(model, train_loader, optimizer, criterion, scheduler, device, epoch):
    model.train() #setting the model to training mode
    count = 0 #initialize the variable to count how many inputs processed in one epoch
    sum_losses = 0 #our sum of losses, later to be averaged
    with tqdm(train_loader, unit="batch") as tepoch: #used to iterate over the dataloader with a progress bar
        for (inputs, labels) in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs) #the output of the model for our input
            loss = criterion(outputs, labels) #calculating the loss
            sum_losses += loss.item()
            count += 1
            loss.backward() #calculating the gradient
            optimizer.step() #doing the backprop over the loss using the calculated gradient
            optimizer.zero_grad() #deleting the cache
            tepoch.set_postfix(loss=loss.item()) #appending the loss to the progress bar
            sleep(0.1) #to avoid overlap of contiguous progress bars
    return sum_losses/count #return the average loss

def valid_fn(model, valid_loader, criterion, device, epoch):
    model.eval()
    preds = []
    sum_losses = 0
    count = 0
    with tqdm(valid_loader, unit="batch") as tepoch:
        for (inputs, labels) in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = model(inputs)
            loss = criterion(outputs, labels)
            sum_losses += loss.item()
            count += 1
            preds.append(outputs.softmax(1).to('cpu').numpy()) #predictions calculated and converted to numpy (via cpu)
            tepoch.set_postfix(loss=loss.item())
            sleep(0.1)
    predictions = np.concatenate(preds)
    return sum_losses/count, predictions

# Our main training loop

In [20]:
def train_loop():
    train_dataset = CustomDataset(train_data, transform=get_transform(data='train')) #training dataset class
    valid_dataset = CustomDataset(valid_data, transform=get_transform(data='valid')) #validation dataset class
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, drop_last=True) #training dataloader
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, drop_last=False) #validation dataloader
    for model_name in CFG.model_names: #iterating over the models listed in CFG
        model = Model(model_name) #creating model
        model.to(device)
        #next 3 lines are creating the utils
        optimizer = get_optimizer(model)
        criterion = get_criterion()
        scheduler = get_scheduler(optimizer)
        best_score = -1 #initializing the score variable
        best_loss = np.inf #initializing the loss variable
        print('*'*25)
        print(f"Model : {model_name}")
        print('*'*25)
        print()
        for epoch in range(CFG.n_epochs):
            avg_loss = train_fn(model, train_loader, optimizer, criterion, scheduler, device, epoch) #training loss
            avg_val_loss, predictions = valid_fn(model, valid_loader, criterion, device, epoch) #validation loss and predictions
            valid_labels = valid_data["Class"].to_numpy().reshape(len(predictions), ) #getting the validation labels
            print(np.argmax(predictions,axis=1))
            scheduler.step()
            score = get_score(valid_labels, np.argmax(predictions, axis=1)) #evaluating the validation labels and predictions
            print(f"Train Loss = {avg_loss}\nVal Loss = {avg_val_loss}\nScore = {score}")
            #saving the model if score exceeds previous best score
            if score > best_score: 
                print("Score Improved")
                best_score = score
                print(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f}')
                torch.save({'model': model.state_dict(),
                            'preds': predictions,
                            'optimizer': optimizer.state_dict(),
                            'scheduler': scheduler.state_dict()},
                            './'+f'{model_name}.pth')
        check_point = torch.load('./'+f'{model_name}.pth')
        valid_data['preds'] = check_point['preds'].argmax(1)
    return valid_data

# Main funtion

In [21]:
def main():
    if CFG.train:
        df = train_loop()

## Calling the main function

In [22]:
if __name__ == '__main__':
    main()

model.safetensors:   0%|          | 0.00/432M [00:00<?, ?B/s]

head.fc
Adjusting learning rate of group 0 to 1.0000e-03.
*************************
Model : regnetx_320.tv2_in1k
*************************



  0%|          | 0/105 [00:00<?, ?batch/s]

  0%|          | 0/28 [00:00<?, ?batch/s]

[6 4 3 ... 3 3 3]
Adjusting learning rate of group 0 to 1.0000e-03.
Train Loss = 2.0587709290640697
Val Loss = 1.2944096411977495
Score = 0.5111803000283045
Score Improved
Epoch 1 - Save Best Score: 0.5112


  0%|          | 0/105 [00:00<?, ?batch/s]

KeyboardInterrupt: 